In [1]:
import os
import json

os.chdir('../')
os.getcwd()

'/workspace'

In [68]:
from augmentation import RoMixGen_Img, RoMixGen_Txt
import random

In [69]:
RoMixGen_Img = RoMixGen_Img(image_dict = json.load(open('data/COCO/Annotations/coco_objbg_info_0.01_bbox_center_0.7.json')),
                            image_root = 'data/COCO/Images_0.01_bbox_center_0.7',
                            transform_after_mix = None,
                            resize_ratio = 2)
RoMixGen_Txt = RoMixGen_Txt(image_caption = json.load(open('data/COCO/Annotations/coco_objbg_info_0.01_bbox_center_0.7.json')))

In [5]:
with open ('data/COCO/Annotations/coco_objbg_info_0.01_bbox_center_0.7.json') as f:
    objbg_dict = json.load(f)
with open ('data/COCO/Annotations/coco_img_info_0.01_bbox_center_0.7.json') as f:
    image_dict = json.load(f)
with open ('data/COCO/Annotations/coco.json') as f:
    original_dataset = json.load(f)

In [74]:
# make 123,287 romixed images and captions
# new image id start from 000001 ~ 123287
# save image to data/COCO/pre_romix_coco_0.01_bbox_center_0.7
# save caption to data/COCO/Anotations/pre_romix_caption_0.01_bbox_center_0.7.json
total_imagenum = 123287
obj_ids = objbg_dict['obj']
bg_ids = objbg_dict['bg']

In [9]:
for img_id in range (total_imagenum):
    # random pop
    obj_id = random.choiceobj_ids

dict_keys(['obj', 'unusable_bg', 'bg'])

In [62]:
back_translate(bg_new)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'A group of bike chairs stopped in front of a building, a group of chairs on bicy in front of a church., cyclists on the corner outside a church., children on bicycles outside a white church., several chairs on bicycles in front of a building.]'

In [55]:
random.choice(obj_cats)

'furniture'